# **СОРЕВНОВАНИЕ 2**

## 1. Импорты

In [ ]:
import json
from pathlib import Path

import pandas as pd
import os
from PIL import Image

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split

## 2. Конфигурация 

In [2]:
DATA_ROOT = Path("splitted")
TRAIN_DIR = DATA_ROOT / "train"
TEST_DIR = DATA_ROOT / "test"
LABELS_JSON = Path("labels.json")
TRAIN_ANN = Path("train_annotations.csv")
SUBMISSION_TEMPLATE = Path("submission.csv")

OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Device: cpu


## 3. Загрузка меток

In [3]:
with open(LABELS_JSON, "r", encoding="utf-8") as f:
    label_map = json.load(f)
# inverse map name->index
name_to_idx = {v: int(k) for k, v in label_map.items()}
idx_to_name = {int(k): v for k, v in label_map.items()}
NUM_CLASSES = len(idx_to_name)
print("Classes:", NUM_CLASSES)

Classes: 18


## 4. Dataset class

In [ ]:
class OnePieceDataset(Dataset):
    def __init__(self, items, transform=None):
        self.items = items
        self.transform = transform

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        path, label = self.items[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        if label is None:
            return img, os.path.basename(path)
        return img, label

## 5. Формирование списка файлов

In [5]:
train_items = []
for cls_name, cls_idx in name_to_idx.items():
    cls_dir = TRAIN_DIR / cls_name
    if not cls_dir.exists():
        print("Warning: missing", cls_dir)
        continue
    for p in cls_dir.iterdir():
        if p.suffix.lower() in [".jpg", ".jpeg", ".png"]:
            train_items.append((str(p), cls_idx))

print("Train items:", len(train_items))

# stratified split
paths = [p for p, lbl in train_items]
labels = [lbl for p, lbl in train_items]
train_paths, val_paths, train_labels, val_labels = train_test_split(
    paths, labels, test_size=0.15, random_state=42, stratify=labels
)
train_list = list(zip(train_paths, train_labels))
val_list = list(zip(val_paths, val_labels))
print("Train:", len(train_list), "Val:", len(val_list))

Train items: 2915
Train: 2477 Val: 438


## 6. Аугментации 

In [6]:
IMG_SIZE = 224
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.1)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

test_transform = val_transform

## 7. Dataloaders

In [7]:
BATCH = 32
train_ds = OnePieceDataset(train_list, transform=train_transform)
val_ds = OnePieceDataset(val_list, transform=val_transform)
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=4, pin_memory=True)

## 8. ResNet18 (pretrained)

In [8]:
model = models.resnet18(pretrained=True)
# заменим final fc
in_f = model.fc.in_features
model.fc = nn.Linear(in_f, NUM_CLASSES)
model = model.to(DEVICE)
print(model)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\lenya/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


c:\Users\lenya\anaconda3\envs\spbu-dl\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lenya\anaconda3\envs\spbu-dl\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:04<00:00, 10.9MB/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 9. Функция обучения и валидации

In [9]:
from tqdm import tqdm

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    pbar = tqdm(loader, desc="Train")
    for imgs, labels in pbar:
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        preds = out.argmax(dim=1)
        running_corrects += (preds == labels).sum().item()
        total += imgs.size(0)
        pbar.set_postfix(loss=running_loss/total, acc=running_corrects/total)
    return running_loss/total, running_corrects/total

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Val"):
            imgs = imgs.to(device)
            labels = labels.to(device)
            out = model(imgs)
            loss = criterion(out, labels)
            running_loss += loss.item() * imgs.size(0)
            preds = out.argmax(dim=1)
            running_corrects += (preds == labels).sum().item()
            total += imgs.size(0)
    return running_loss/total, running_corrects/total

## 10. Основной цикл

In [ ]:
EPOCHS = 12
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

best_acc = 0.0
history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch+1}/{EPOCHS} ===")
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, DEVICE)
    val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)
    print(f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | Val loss {val_loss:.4f}, acc {val_acc:.4f}")
    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    # scheduler step by val_acc
    scheduler.step(val_acc)

    # save best
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), OUTPUT_DIR / "best_resnet18.pth")
        print("Saved best model.")
print("Best val acc:", best_acc)


=== Epoch 1/12 ===


Train:   0%|          | 0/78 [00:00<?, ?it/s]c:\Users\lenya\anaconda3\envs\spbu-dl\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


## 11. Финальные шаги

In [ ]:
test_items = []
for p in sorted(TEST_DIR.iterdir()):
    if p.suffix.lower() in [".jpg", ".jpeg", ".png", ".png"]:
        test_items.append(str(p))
print("Test items:", len(test_items))

test_ds = OnePieceDataset([(p, None) for p in test_items], transform=test_transform)
test_loader = DataLoader(test_ds, batch_size=BATCH, shuffle=False, num_workers=4)

# load best model
model.load_state_dict(torch.load(OUTPUT_DIR / "best_resnet18.pth", map_location=DEVICE))
model.eval()

preds_list = []
fnames = []
with torch.no_grad():
    for imgs, names in tqdm(test_loader, desc="Test"):
        imgs = imgs.to(DEVICE)
        out = model(imgs)
        preds = out.argmax(dim=1).cpu().numpy()
        preds_list.extend(preds.tolist())
        fnames.extend(names)

sub_df = pd.read_csv(SUBMISSION_TEMPLATE)
pred_map = {Path(fn).stem: p for fn, p in zip(fnames, preds_list)}
# submission
out_labels = []
for idx, row in sub_df.iterrows():
    key = row['id']
    if key in pred_map:
        out_labels.append(pred_map[key])
    else:
        # fallback
        out_labels.append(int(0))
sub_df['label'] = out_labels
sub_df.to_csv(OUTPUT_DIR / "submission_resnet18.csv", index=False)
print("Saved submission to", OUTPUT_DIR / "submission_resnet18.csv")